In [47]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import pandas as pd
import numpy as np
import sqlite3
import time
pd.options.display.float_format = '{:,.1f}'.format

In [48]:
# 드라이버 객체 생성
options = webdriver.ChromeOptions()
#options.add_argument('headless')
#options.add_argument('diable-gpu')
driver = webdriver.Chrome('chromedriver.exe', chrome_options=options)

In [83]:
# 링크 주소 가져오기# 링크 주소 
def getData(n): 
    row = []
    if n < 50:
        selector = ['tr[data-index="'+str(n)+'"]','td.inf','a.newLink._link']
        selector = ' > '.join(selector)
        x = driver.find_element_by_css_selector(selector).get_attribute('href')
        row.append(x)
        
        selector = ['tr[data-index="'+str(n)+'"]','td.inf','a.newLink._link','span.cls','strong']
        selector = ' > '.join(selector)
        x = driver.find_element_by_css_selector(selector).text
        row.append(x)
        
        selector = ['tr[data-index="'+str(n)+'"]','td.inf','a.newLink._link','span.cls','em']
        selector = ' > '.join(selector)
        x = driver.find_element_by_css_selector(selector).text
        row.append(x)
        
        selector = ['tr[data-index="'+str(n)+'"]','td.inf','a.newLink._link','span.dtl','strong']
        selector = ' > '.join(selector)
        x = driver.find_element_by_css_selector(selector).text
        row.append(x)
        
        selector = ['tr[data-index="'+str(n)+'"]','td.inf','a.newLink._link','span.dtl','em']
        selector = ' > '.join(selector)
        x = driver.find_element_by_css_selector(selector).text
        row.append(x)
        
        selector = ['tr[data-index="'+str(n)+'"]','td.inf','span.detail','span.trs']
        selector = ' > '.join(selector)
        x = driver.find_element_by_css_selector(selector).text
        row.append(x)
        
        selector = ['tr[data-index="'+str(n)+'"]','td.inf','span.detail','span.fue']
        selector = ' > '.join(selector)
        x = driver.find_element_by_css_selector(selector).text
        row.append(x)
        
        selector = ['tr[data-index="'+str(n)+'"]','td.yer']
        selector = ' > '.join(selector)
        x = driver.find_element_by_css_selector(selector).text
        row.append(x)
        
        selector = ['tr[data-index="'+str(n)+'"]','td.km']
        selector = ' > '.join(selector)
        x = driver.find_element_by_css_selector(selector).text
        row.append(x)
        
        selector = ['tr[data-index="'+str(n)+'"]','td.prc']
        selector = ' > '.join(selector)
        x = driver.find_element_by_css_selector(selector).text
        row.append(x)
        
        selector = ['tr[data-index="'+str(n)+'"]','td.dat_loc','span:nth-of-type(1)']
        selector = ' > '.join(selector)
        x = driver.find_element_by_css_selector(selector).text
        row.append(x)
        
        selector = ['tr[data-index="'+str(n)+'"]','td.dat_loc','span:nth-of-type(2)']
        selector = ' > '.join(selector)
        x = driver.find_element_by_css_selector(selector).text
        row.append(x)
        
        
        return row
    else:
        return None
    
# 페이지 클릭하기
# 아무 페이지나 실행되진 않고 순차적으로 실행해야 함
def clickPage(page):
    selector = ['a[data-page="'+str(page)+'"]']
    selector = ' > '.join(selector)
    driver.find_element_by_css_selector(selector).click()

In [66]:
# 데이터 적재할 DB 생성
conn = sqlite3.connect('skencar.db')
cur = conn.cursor()

In [94]:
# 데이터 적재할 테이블 생성
cur.execute('drop table cars')
cur.execute('create table cars(href,cls1,cls2,dtl1,dtl2,trs,fue,yer,km,prc,dat_loc1,dat_loc2)')
cur.execute('delete from cars')

In [93]:
# 홈페이지 메인 화면으로 접속
url = 'http://www.encar.com/index.do'
driver.get(url)

# 크롤링 메인 화면으로 접속
driver.find_element_by_id('manufact').click()
selector = ['#manufactListText','ul:nth-of-type(2)','li','a']
selector = ' > '.join(selector)
driver.find_element_by_css_selector(selector).click()
driver.execute_script("goSearch('car')")
time.sleep(2)
Select(driver.find_element_by_id('pagerow')).select_by_value('50')

In [ ]:
# 링크 긁어오기
pages = 111
for page in range(1,pages):
    if page != 1: clickPage(page)
    time.sleep(0.5)
    for i in range(50):
        cur.execute('insert into cars values (?,?,?,?,?,?,?,?,?,?,?,?)', getData(i))
    if page%10 == 1:
        print('--- {:3} Page... done ---'.format(page))
        conn.commit()

---   1 Page... done ---


In [ ]:
# 적재한 데이터 출력
cur.execute('select * from cars')
rows = cur.fetchall()
df = pd.DataFrame(rows)